In [1]:
import numpy as np
import pandas as pd

# Linear models
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet

# Support Vector Machines
from sklearn.svm import SVR, LinearSVR

# Tree-based models
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import (
    RandomForestRegressor,
    ExtraTreesRegressor,
    GradientBoostingRegressor,
    AdaBoostRegressor,
    HistGradientBoostingRegressor
)

# Neighbors
from sklearn.neighbors import KNeighborsRegressor

# Neural networks
from sklearn.neural_network import MLPRegressor

# Gaussian Processes
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF

# Gradient boosting libraries outside sklearn
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor


from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler, OrdinalEncoder
from sklearn.compose import ColumnTransformer
import category_encoders as ce

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import mean_absolute_error

from sklearn.decomposition import PCA

import warnings
warnings.filterwarnings('ignore')

In [2]:
# Load the stage 6 property data
df = pd.read_csv(r'Output Files\stage_6_property_data.csv') 
df.head(3)

,City,Area,Type of Property,Transaction Type,Property Lifespan,Commercial,Covered Area,Bedrooms,Bathrooms,Balconies,...,Water Storage,Vaastu Compliant,Visitor Parking,Intercom Facility,Maintenance Staff,Internet/Wi-Fi Connectivity,DTH Television Facility,Piped Gas,Jogging and Strolling Track,Price (Crores)
0,Thane,Kalyan West,Apartment,New Property,New construction,Y,763.571943,2,2.0,2.0,...,1,1,1,1,1,1,1,1,1,0.63
1,Thane,Kalyan West,Apartment,New Property,New construction,N,850.000000,2,2.0,2.0,...,1,1,1,1,1,1,1,1,1,0.54
2,Thane,Kalyan West,Apartment,New Property,Less than 5 years,N,1050.000000,2,2.0,3.0,...,1,1,1,1,1,1,1,1,1,0.90


In [3]:
# Check shape
df.shape

(6280, 47)

-----

### Nature of Categorical features:

In [4]:
# Print Data type of features
df.dtypes

City                               object
Area                               object
Type of Property                   object
Transaction Type                   object
Property Lifespan                  object
Commercial                         object
Covered Area                      float64
Bedrooms                            int64
Bathrooms                         float64
Balconies                          object
House Help Room                     int64
Store Room                          int64
Puja Room                           int64
Study                               int64
dist_to_csia_airport_km           float64
dist_to_thane_station_km          float64
dist_to_andheri_station_km        float64
dist_to_csmt_station_km           float64
dist_to_nariman_point_km          float64
dist_to_andheri_east_comm_km      float64
dist_to_marine_drive_km           float64
dist_to_navi_mumbai_airport_km    float64
dist_to_vashi_station_km          float64
dist_to_sanjay_gandhi_np_km       

- Transaction Type: Nominal feature with no ordering.
- Type of Property: Nominal feature with no ordering.
- City: Nominal feature with no ordering.
- Area:  Nominal feature with no ordering. [High Cardinality: 53 unique values]
- Commercial: Nominal feature with no ordering.
- Furnished Type: Feature with inherent ordering.
- Balconies: Feature with inherent ordering.
- Facing: Nominal feature with no ordering.
- Floor Level: Feature with inherent ordering.
- Property Lifespane: Feature with inherent ordering.
- Luxury Category: Feature with inherent ordering.

------------

### Utilities

In [5]:
def scorer(X, y_transformed, model_name: str, model, preprocessor):
    '''
    This function builds a pipeline with a given regression model and a preconfigured preprocessor, then evaluates it using both R² score (via 10-fold cross-validation) and Mean Absolute Error (MAE) on a 
    hold-out test set.

    Args:
        model_name : Name of the regression model

        model : An instantiated regression model
    '''
    
    output = []
    
    output.append(model_name)
    
    pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('regressor', model)
    ])
    
    # K-fold cross-validation
    kfold = KFold(n_splits=10, shuffle=True, random_state=42)
    scores = cross_val_score(pipeline, X, y_transformed, cv=kfold, scoring='r2')
    
    # Average R2 score
    output.append(scores.mean())
    
    # For MAE
    X_train, X_test, y_train, y_test = train_test_split(X, y_transformed, test_size=0.2, random_state=42)
    
    pipeline.fit(X_train, y_train)
    
    y_pred = pipeline.predict(X_test)
    
    output.append(mean_absolute_error(np.expm1(y_test), np.expm1(y_pred)))
    
    return output
    

In [6]:
# ML models
model_dict = {
    # Linear Models
    'LinearRegression': LinearRegression(),
    'Ridge': Ridge(),
    'Lasso': Lasso(),
    'ElasticNet': ElasticNet(),

    # Support Vector Machines
    'SVR': SVR(),
    'LinearSVR': LinearSVR(),

    # Tree-Based Models
    'DecisionTreeRegressor': DecisionTreeRegressor(),
    'RandomForestRegressor': RandomForestRegressor(),
    'ExtraTreesRegressor': ExtraTreesRegressor(),

    # Boosting Algorithms
    'GradientBoostingRegressor': GradientBoostingRegressor(),   # shallow trees as base learners
    'HistGradientBoostingRegressor': HistGradientBoostingRegressor(),
    'AdaBoostRegressor': AdaBoostRegressor(),
    'XGBRegressor': XGBRegressor(),
    'LGBMRegressor': LGBMRegressor(verbose=-1),
    'CatBoostRegressor': CatBoostRegressor(verbose=0),

    # Nearest Neighbors
    'KNeighborsRegressor': KNeighborsRegressor(),

    # Neural Networks
    'MLPRegressor': MLPRegressor(),

    # Gaussian Processes
    'GaussianProcessRegressor': GaussianProcessRegressor(kernel=RBF())
}


# Strategy
As the Machine learning algorithms only understands numbers, the categorical columns with object/str dtype needs to be transformed into numbers. For this 3 different encoding techniques will be deployed. The best performing combination of ML algorithm + encoding technique will finally be deployed on production for predicting prices of the property.

- Ordinal encoding: Ordinal encoding works well for tree-based models, but not for linear/distance-based models due to artificial ordering.

- One-hot encoding: One-hot encoding works well for linear and distance-based models, watch for high cardinality.

- Target encoding: Target encoding can be powerful for tree-based models with high cardinality features, but must be used with strict data leakage precautions.

------------------

In [7]:
# Independent features
X = df.drop(columns=['Price (Crores)'])

# Dependent feature [Target Variable]
y = df['Price (Crores)']

In [8]:
# Log transform y
y_transformed = np.log1p(y)

#### Strategy 1: Ordinal encoding

In [12]:
# Define specific ordering for each ordinal column
ordinal_categories = [
    list(df['City'].value_counts(ascending=True).index), # City → based on frequency
    list(df['Area'].value_counts(ascending=True).index), # Area → based on frequency
    list(df['Type of Property'].value_counts(ascending=True).index), # Type of Property → based on frequency
    list(df['Transaction Type'].value_counts(ascending=True).index), # Transaction Type → based on frequency
    ['New construction', 'Less than 5 years', '5 to 10 years', '10 to 15 years', '15 to 20 years', 'Above 20 years'],  # Property Lifespan
    list(df['Commercial'].value_counts(ascending=True).index), # Commercial → based on frequency
    ['1.0', '2.0', '3.0', '3+'],  # Balconies
    ['Unfurnished', 'Semi-Furnished', 'Furnished'],  # Furnished Type
    ['Low rise (< 5)', 'Mid rise (5 to 10)', 'High rise (> 10)'],  # Floor Level
    list(df['Facing'].value_counts(ascending=True).index), # Facing → based on frequency
]

In [ ]:
# Column transformer for preprocessing
preprocessor_for_ordinal_encoding = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['Covered Area', 'Bedrooms', 'Bathrooms', 'dist_to_csia_airport_km', 'dist_to_thane_station_km', 'dist_to_andheri_station_km', 'dist_to_csmt_station_km', \
                                   'dist_to_nariman_point_km', 'dist_to_andheri_east_comm_km', 'dist_to_marine_drive_km', 'dist_to_navi_mumbai_airport_km', 'dist_to_vashi_station_km', \
                                   'dist_to_sanjay_gandhi_np_km', 'dist_to_phoenix_mall_kurla_km', 'dist_to_kokilaben_hospital_km', 'dist_to_dh_ambani_school_km']),
        
        ('cat', Pipeline([
            ('ord_enc', OrdinalEncoder(categories=ordinal_categories)),
            ('scaler', StandardScaler())  # Scale ordinals for balanced influence
        ]), ['City', 'Area', 'Type of Property', 'Transaction Type', 'Property Lifespan', 'Commercial', 'Balconies', 'Furnished Type', 'Floor Level', 'Facing'])
    ], 
    remainder='passthrough'
)

In [14]:
# Train models
model_output_ordinal = []
for model_name, model in model_dict.items():
    model_output_ordinal.append(scorer(X, y_transformed, model_name, model, preprocessor_for_ordinal_encoding))

In [15]:
# Results Dataframe
model_df = pd.DataFrame(
    model_output_ordinal,
    columns=['Algorithm','R2 Score','MAE']
).sort_values('MAE').reset_index(drop=True)

# Display
model_df

,Algorithm,R2 Score,MAE
0,CatBoostRegressor,0.936466,0.425996
1,RandomForestRegressor,0.926691,0.437446
2,HistGradientBoostingRegressor,0.932027,0.445436
3,LGBMRegressor,0.931204,0.451013
4,ExtraTreesRegressor,0.924233,0.461776
5,XGBRegressor,0.928338,0.465880
6,GradientBoostingRegressor,0.919450,0.481677
7,SVR,0.899210,0.545182
8,MLPRegressor,0.886290,0.589778
9,DecisionTreeRegressor,0.861047,0.611569


##### Result:
- CatBoostRegressor performs the best with R2 score of 0.93 impyling it is able to explain ~93% variance in Price of the properties and mean absolute error of 0.43 Crores.

- As expected the Linear models: Linear regression, Ridge , Lasso and ElasticNet did not perform well as they do not go well with ordinal encoding for nominal categories with no inherent ordering. 

- As expected the tree based models performed good with Ordinal encoding.

------------------

#### Strategy 2: One-hot encoding
- Apply one hot encoding to those categorical columns where there is no inherent ordering present.

- Problem with one hot encoding: If high cardinality column is present then the a large number of columns would be added leading to high dimensionality and sparse data. Sparse data can work fine for some algorithms (like linear models), but others (like decision trees) might struggle if the dataset is huge and sparse. Dimensionality reduction techniques can be considered here to reduce training time (like OneHotEncoding with PCA).
    
- Performance of linear models like linear regression should improve after using OneHotEncoding.

In [18]:
# Define specific ordering for each ordinal column
ordinal_categories = [
    ['New construction', 'Less than 5 years', '5 to 10 years', '10 to 15 years', '15 to 20 years', 'Above 20 years'],  # Property Lifespan
    ['1.0', '2.0', '3.0', '3+'],  # Balconies
    ['Unfurnished', 'Semi-Furnished', 'Furnished'],  # Furnished Type
    ['Low rise (< 5)', 'Mid rise (5 to 10)', 'High rise (> 10)'],  # Floor Level    
]

In [ ]:
# Column transformer for preprocessing
preprocessor_for_onehot_ordinal_encoding = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['Covered Area', 'Bedrooms', 'Bathrooms', 'dist_to_csia_airport_km', 'dist_to_thane_station_km', 'dist_to_andheri_station_km', 'dist_to_csmt_station_km', \
                                   'dist_to_nariman_point_km', 'dist_to_andheri_east_comm_km', 'dist_to_marine_drive_km', 'dist_to_navi_mumbai_airport_km', 'dist_to_vashi_station_km', \
                                   'dist_to_sanjay_gandhi_np_km', 'dist_to_phoenix_mall_kurla_km', 'dist_to_kokilaben_hospital_km', 'dist_to_dh_ambani_school_km']),

        ('cat_nom', OneHotEncoder(drop='first', sparse_output=False), ['City', 'Area', 'Type of Property', 'Transaction Type', 'Commercial', 'Facing']),  # Drop to remove multicollinearity
        
        ('cat_ord', Pipeline([
            ('ord_enc', OrdinalEncoder(categories=ordinal_categories)),
            ('scaler', StandardScaler())  # Scale ordinals for balanced influence
        ]), ['Property Lifespan', 'Balconies', 'Furnished Type', 'Floor Level'])       
    ], 
    remainder='passthrough'
)

In [21]:
# Train models
model_output_onehot_ordinal = []
for model_name, model in model_dict.items():
    model_output_onehot_ordinal.append(scorer(X, y_transformed, model_name, model, preprocessor_for_onehot_ordinal_encoding))

In [22]:
# Results Dataframe
model_df = pd.DataFrame(
    model_output_onehot_ordinal,
    columns=['Algorithm','R2 Score','MAE']
).sort_values('MAE').reset_index(drop=True)

# Display
model_df

,Algorithm,R2 Score,MAE
0,CatBoostRegressor,0.940718,0.418529
1,XGBRegressor,0.934328,0.434279
2,ExtraTreesRegressor,0.932172,0.436338
3,HistGradientBoostingRegressor,0.933287,0.438394
4,LGBMRegressor,0.933526,0.445844
5,RandomForestRegressor,0.927986,0.446085
6,SVR,0.932606,0.458551
7,GradientBoostingRegressor,0.924562,0.474350
8,MLPRegressor,0.924408,0.482193
9,Ridge,0.923155,0.495706


#### Result:
- CatBoostRegressor remains the best performing algorithm with R2 score of 0.94. The mean absolute error has further reduced down to 0.42 crores.

- High-cardinality one-hot feature 'Area' impact on tree-based models:

    - The presence of a high-cardinality feature like Area exploded the feature space after one-hot encoding, with most added columns being zero for any given row, resulting in sparse data.

    - Single Decision Tree:

        - One tree tries to fit all the data by itself. Overfits rare categories because each unique one-hot column can create splits that perfectly match a few rows.

        - Fragments the data into very small subsets, reducing statistical power in leaves.

        - Wastes tree depth exploring mostly-zero irrelevant features.
        
        - Lacks any averaging mechanism, so high variance leads to poor generalization.

    - AdaBoost and Gradient Boosting (with shallow trees):

        - Typically use shallow base learners (depth-1 stumps or shallow trees).

        - Cannot capture complex patterns in high-dimensional sparse one-hot data.

        - Still overfits rare categories quickly due to many sparse column, leading to memorization rather than generalization.

        - Lack the averaging effect of Random Forest, so variance remains high and performance drops.
    

- On contarary why some tree ensembles still performed well?:

    - Tree-based models like Random Forest, Extra Trees, and XGBoost are more robust to high-cardinality one-hot features.

    - Random Forest / Extra Trees build many deep trees on bootstrapped subsets of data and features. Even if some splits overfit rare one-hot columns, averaging across hundreds of trees reduces variance.

    - XGBoost (Extreme Gradient Boosting) with deep trees handles high-dimensional sparse input better than AdaBoost with shallow stumps.

- Summary of boosting/tree behavior:

    - Shallow boosting + high-cardinality one-hot → performance drop

    - Deep ensemble trees + high-cardinality one-hot → performance usually stable

- Impact on linear models:

    - Linear models like Linear Regression and Ridge Regression improved as expected with one-hot encoding of nominal features.

    - These models just multiply each feature by a weight and sum them, so sparse columns (mostly zeros) are effectively ignored, making them robust to high-dimensional one-hot features.


--------------

#### Strategy 3: OneHotEncoding with PCA
The presence of a high-cardinality categorical feature like Area creates many sparse columns after one-hot encoding, which can cause tree-based models without averaging, such as a single Decision Tree, as well as boosting methods like AdaBoost and Gradient Boosting with shallow trees, to overfit and struggle to generalize. Applying PCA or other dimensionality reduction techniques in this scenario can help reduce sparsity and improve the performance of these models.

In [23]:
# Define specific ordering for each ordinal column
ordinal_categories = [
    ['New construction', 'Less than 5 years', '5 to 10 years', '10 to 15 years', '15 to 20 years', 'Above 20 years'],  # Property Lifespan
    ['1.0', '2.0', '3.0', '3+'],  # Balconies
    ['Unfurnished', 'Semi-Furnished', 'Furnished'],  # Furnished Type
    ['Low rise (< 5)', 'Mid rise (5 to 10)', 'High rise (> 10)'],  # Floor Level
]

In [ ]:
# Column transformer for preprocessing
preprocessor_for_onehotPCA_ordinal_encoding = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['Covered Area', 'Bedrooms', 'Bathrooms', 'dist_to_csia_airport_km', 'dist_to_thane_station_km', 'dist_to_andheri_station_km', 'dist_to_csmt_station_km', \
                                   'dist_to_nariman_point_km', 'dist_to_andheri_east_comm_km', 'dist_to_marine_drive_km', 'dist_to_navi_mumbai_airport_km', 'dist_to_vashi_station_km', \
                                   'dist_to_sanjay_gandhi_np_km', 'dist_to_phoenix_mall_kurla_km', 'dist_to_kokilaben_hospital_km', 'dist_to_dh_ambani_school_km']),

        ('cat_nom_low_cardinality', OneHotEncoder(drop='first', sparse_output=False), ['City', 'Type of Property', 'Transaction Type', 'Commercial']),  # Drop to remove multicollinearity
        
        ('cat_nom_high_cardinality', Pipeline([
            ('one-hot', OneHotEncoder(drop='first',sparse_output=False)),  # sparse_output=False outputs a dense NumPy array (all 0/1 stored explicitly), as next pipeline step is PCA that requires a dense array as input (PCA in scikit-learn does not accept sparse matrices.)
            ('pca', PCA(n_components=0.95))  # PCA after one-hot encoding of high cardinality features
            ]), ['Area', 'Facing']), 
        
        ('cat_ord', Pipeline([
            ('ord_enc', OrdinalEncoder(categories=ordinal_categories)),
            ('scaler', StandardScaler())  # Scale ordinals for balanced influence
        ]), ['Property Lifespan', 'Balconies', 'Furnished Type', 'Floor Level'])       
    ], 
    remainder='passthrough'
)

In [26]:
# Train models
model_output_onehotPCA_ordinal = []
for model_name, model in model_dict.items():
    model_output_onehotPCA_ordinal.append(scorer(X, y_transformed, model_name, model, preprocessor_for_onehotPCA_ordinal_encoding))

In [27]:
# Results Dataframe
model_df = pd.DataFrame(
    model_output_onehotPCA_ordinal,
    columns=['Algorithm','R2 Score','MAE']
).sort_values('MAE').reset_index(drop=True)

# Display
model_df

,Algorithm,R2 Score,MAE
0,CatBoostRegressor,0.938804,0.425918
1,ExtraTreesRegressor,0.930505,0.431602
2,HistGradientBoostingRegressor,0.932805,0.443636
3,RandomForestRegressor,0.926626,0.446326
4,LGBMRegressor,0.933695,0.447514
5,XGBRegressor,0.931140,0.449569
6,GradientBoostingRegressor,0.926459,0.470546
7,SVR,0.927599,0.470852
8,MLPRegressor,0.918588,0.489222
9,LinearSVR,0.908603,0.511637


#### Result:
- The performance of Gradient Boosting has slightly improved.
    - Gradient Boosting builds trees sequentially on residuals. Even with the same depth, trees now operate on denser, lower-dimensional features, which reduces overfitting to rare categories and allows the boosting process to focus on meaningful variance.

- Single Decison Tree still perform poorly as it still has no averaging, so high variance remains — it overfits whatever patterns remain in the PCA-transformed data.

- AdaBoost with shallow learners still cannot model complex interactions in the reduced feature space and hence performs poorly.

- The overall best performance still remains around 0.93 with an MAE of 0.43 for CatBoostRegressor.

--------------------

#### Strategy 4: Target encoding for High cardinality feature

- Target encoding is useful for columns with very high cardinality.

- Encoding is done on the basis of target variable.

- Caution! Target encoding can result in Data leakage problem. First peform train-test split and then do target encoding to prevent data leakage problem. In our case we are safe as we are doing cross-validation.

- Goes well with tree based models. Results are not that good with linear models (but better than ordinal encoding).

In [28]:
# Define specific ordering for each ordinal column
ordinal_categories = [
    ['New construction', 'Less than 5 years', '5 to 10 years', '10 to 15 years', '15 to 20 years', 'Above 20 years'],  # Property Lifespan
    ['1.0', '2.0', '3.0', '3+'],  # Balconies
    ['Unfurnished', 'Semi-Furnished', 'Furnished'],  # Furnished Type
    ['Low rise (< 5)', 'Mid rise (5 to 10)', 'High rise (> 10)'],  # Floor Level
]

In [ ]:
# Column transformer for preprocessing
preprocessor_for_onehotpca_ordinal_target_encoding = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['Covered Area', 'Bedrooms', 'Bathrooms', 'dist_to_csia_airport_km', 'dist_to_thane_station_km', 'dist_to_andheri_station_km', 'dist_to_csmt_station_km', \
                                   'dist_to_nariman_point_km', 'dist_to_andheri_east_comm_km', 'dist_to_marine_drive_km', 'dist_to_navi_mumbai_airport_km', 'dist_to_vashi_station_km', \
                                   'dist_to_sanjay_gandhi_np_km', 'dist_to_phoenix_mall_kurla_km', 'dist_to_kokilaben_hospital_km', 'dist_to_dh_ambani_school_km']),

        ('cat_nom_low_cardinality', OneHotEncoder(drop='first', sparse_output=False), ['City', 'Type of Property', 'Transaction Type', 'Commercial']),  # Drop to remove multicollinearity
        
        ('cat_nom_medium_cardinality', Pipeline([
            ('one-hot', OneHotEncoder(drop='first',sparse_output=False)),  # sparse_output=False outputs a dense NumPy array (all 0/1 stored explicitly), as next pipeline step is PCA that requires a dense array as input (PCA in scikit-learn does not accept sparse matrices.)
            ('pca', PCA(n_components=0.95))  # PCA after one-hot encoding of high cardinality features, keep enough components to explain 95% variance
            ]), ['Facing']),
        
        ('cat_nom_high_cardinality', ce.TargetEncoder(), ['Area']),  # Target encoding applied to high cardinality column
        
        ('cat_ord', Pipeline([
            ('ord_enc', OrdinalEncoder(categories=ordinal_categories)),
            ('scaler', StandardScaler())  # Scale ordinals for balanced influence
        ]), ['Property Lifespan', 'Balconies', 'Furnished Type', 'Floor Level'])       
    ], 
    remainder='passthrough'
)

In [30]:
# Train models
model_output_onehot_ordinal_target = []
for model_name, model in model_dict.items():
    model_output_onehot_ordinal_target.append(scorer(X, y_transformed, model_name, model, preprocessor_for_onehotpca_ordinal_target_encoding))

In [31]:
# Results Dataframe
model_df = pd.DataFrame(
    model_output_onehot_ordinal_target,
    columns=['Algorithm','R2 Score','MAE']
).sort_values('MAE').reset_index(drop=True)

# Display
model_df

,Algorithm,R2 Score,MAE
0,RandomForestRegressor,0.937858,0.402164
1,CatBoostRegressor,0.942227,0.410829
2,ExtraTreesRegressor,0.937152,0.418758
3,HistGradientBoostingRegressor,0.940191,0.419759
4,XGBRegressor,0.936349,0.423752
5,LGBMRegressor,0.940528,0.425196
6,GradientBoostingRegressor,0.936370,0.434262
7,SVR,0.928764,0.482423
8,MLPRegressor,0.920169,0.500765
9,LinearRegression,0.915862,0.513316


#### Results:
RandomForestRegressor has achieved the lowest MAE of 0.40 Crores until now with an R2 score of 0.94, while CatBoostRegressor has consistently appeared as the top performer.

---------------

# Summary:

CatBoostRegressor consistently performed the best with different encoding techniques. Therefore, CatBoostRegressor with One-Hot encoding over nominal categorical features and Target encoding over high cardinal feature is selected for further hyperparameter tuning and to build the final Price predictive model.